In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

import warnings
warnings.filterwarnings("ignore")

 - select only columns that appear in flights_test AND delay time
 - check for null values and duplicates
 - datatype conversion: time and categorical data
 - what columns are actually useful (even if they are in flght_test)
 - distributions??

In [31]:
flights_df = pd.read_csv('data/flights_sample_for_cancelled_classification.csv')
test_df = pd.read_csv('data/flights_test.csv')

In [32]:
flights_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 100000 entries, 0 to 99999
Data columns (total 42 columns):
 #   Column               Non-Null Count   Dtype  
---  ------               --------------   -----  
 0   fl_date              100000 non-null  object 
 1   mkt_unique_carrier   100000 non-null  object 
 2   branded_code_share   100000 non-null  object 
 3   mkt_carrier          100000 non-null  object 
 4   mkt_carrier_fl_num   100000 non-null  int64  
 5   op_unique_carrier    100000 non-null  object 
 6   tail_num             90938 non-null   object 
 7   op_carrier_fl_num    100000 non-null  int64  
 8   origin_airport_id    100000 non-null  int64  
 9   origin               100000 non-null  object 
 10  origin_city_name     100000 non-null  object 
 11  dest_airport_id      100000 non-null  int64  
 12  dest                 100000 non-null  object 
 13  dest_city_name       100000 non-null  object 
 14  crs_dep_time         100000 non-null  int64  
 15  dep_time          

In [33]:
flights_df.T

,0,1,2,3,4,5,6,7,8,9,...,99990,99991,99992,99993,99994,99995,99996,99997,99998,99999
fl_date,2019-06-02,2019-09-23,2019-06-20,2019-05-18,2018-05-15,2019-02-27,2019-06-29,2019-12-17,2018-11-14,2019-08-12,...,2018-04-18,2018-04-03,2018-10-06,2019-01-25,2019-01-13,2018-06-04,2019-06-23,2019-01-15,2018-01-17,2018-09-09
mkt_unique_carrier,UA,AA,AA,AA,AA,WN,UA,WN,AS,UA,...,AS,AS,DL,UA,DL,DL,DL,AA,WN,DL
branded_code_share,UA_CODESHARE,AA_CODESHARE,AA,AA_CODESHARE,AA_CODESHARE,WN,UA_CODESHARE,WN,AS,UA_CODESHARE,...,AS,AS,DL,UA,DL_CODESHARE,DL_CODESHARE,DL_CODESHARE,AA_CODESHARE,WN,DL_CODESHARE
mkt_carrier,UA,AA,AA,AA,AA,WN,UA,WN,AS,UA,...,AS,AS,DL,UA,DL,DL,DL,AA,WN,DL
mkt_carrier_fl_num,4885,4847,139,5983,5002,1140,4380,2454,1935,3883,...,786,6,2204,1106,4305,6134,3909,3055,38,3894
op_unique_carrier,C5,PT,AA,YV,PT,WN,EV,WN,AS,ZW,...,AS,AS,DL,UA,OO,YX,OO,OO,WN,OO
tail_num,NaN,N646AE,NaN,N959LR,N625AE,N462WN,N13553,N7811F,N855VA,NaN,...,N513AS,N569AS,N374DA,N87513,N889AS,N872RW,N275SY,N778SK,N552WN,N268SY
op_carrier_fl_num,4885,4847,139,5983,5002,1140,4380,2454,1935,3883,...,786,6,2204,1106,4305,6134,3909,3055,38,3894
origin_airport_id,11042,10868,14771,11298,14100,14831,11618,10821,12892,13930,...,14831,12892,14869,14908,14869,11278,11298,14107,12191,12478
origin,CLE,CAE,SFO,DFW,PHL,SJC,EWR,BWI,LAX,ORD,...,SJC,LAX,SLC,SNA,SLC,DCA,DFW,PHX,HOU,JFK


In [34]:
test_columns = test_df.columns
test_columns

Index(['fl_date', 'mkt_unique_carrier', 'branded_code_share', 'mkt_carrier',
       'mkt_carrier_fl_num', 'op_unique_carrier', 'tail_num',
       'op_carrier_fl_num', 'origin_airport_id', 'origin', 'origin_city_name',
       'dest_airport_id', 'dest', 'dest_city_name', 'crs_dep_time',
       'crs_arr_time', 'dup', 'crs_elapsed_time', 'flights', 'distance'],
      dtype='object')

In [35]:
model_df = flights_df[test_columns]
model_df['arr_delay'] = flights_df['arr_delay']
model_df['cancelled_flights'] = flights_df['cancelled']
model_df.head()

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,cancelled_flights
0,2019-06-02,UA,UA_CODESHARE,UA,4885,C5,NaN,4885,11042,CLE,...,EWR,"Newark, NJ",1610,1759,N,109.0,1,404,NaN,1
1,2019-09-23,AA,AA_CODESHARE,AA,4847,PT,N646AE,4847,10868,CAE,...,CLT,"Charlotte, NC",1949,2055,N,66.0,1,88,NaN,1
2,2019-06-20,AA,AA,AA,139,AA,NaN,139,14771,SFO,...,DFW,"Dallas/Fort Worth, TX",1540,2118,N,218.0,1,1464,NaN,1
3,2019-05-18,AA,AA_CODESHARE,AA,5983,YV,N959LR,5983,11298,DFW,...,HOU,"Houston, TX",1630,1740,N,70.0,1,247,NaN,1
4,2018-05-15,AA,AA_CODESHARE,AA,5002,PT,N625AE,5002,14100,PHL,...,DAY,"Dayton, OH",1540,1736,N,116.0,1,477,NaN,1


## 2. check for null values and duplicates

In [36]:
# missing values percent in columns
def missing_values_table(df):
        '''function that checks dataframe for missing values and returns a table with the results'''
        mis_val = df.isnull().sum()
        mis_val_percent = 100 * df.isnull().sum() / len(df)
        mis_val_table = pd.concat([mis_val, mis_val_percent], axis=1)
        mis_val_table_ren_columns = mis_val_table.rename(
        columns = {0 : 'Missing Values', 1 : '% of Total Values'})
        print(mis_val_table_ren_columns)
        
missing_values_table(model_df)

                    Missing Values  % of Total Values
fl_date                          0              0.000
mkt_unique_carrier               0              0.000
branded_code_share               0              0.000
mkt_carrier                      0              0.000
mkt_carrier_fl_num               0              0.000
op_unique_carrier                0              0.000
tail_num                      9062              9.062
op_carrier_fl_num                0              0.000
origin_airport_id                0              0.000
origin                           0              0.000
origin_city_name                 0              0.000
dest_airport_id                  0              0.000
dest                             0              0.000
dest_city_name                   0              0.000
crs_dep_time                     0              0.000
crs_arr_time                     0              0.000
dup                              0              0.000
crs_elapsed_time            

* missing values are negligble so I'm just dropping all of them

In [39]:
model_df['arr_delay'] = model_df['arr_delay'].replace(np.nan, 0)
print(model_df.arr_delay.isnull().sum())

0


In [40]:
model_df = model_df.dropna()
missing_values_table(model_df)

                    Missing Values  % of Total Values
fl_date                          0                0.0
mkt_unique_carrier               0                0.0
branded_code_share               0                0.0
mkt_carrier                      0                0.0
mkt_carrier_fl_num               0                0.0
op_unique_carrier                0                0.0
tail_num                         0                0.0
op_carrier_fl_num                0                0.0
origin_airport_id                0                0.0
origin                           0                0.0
origin_city_name                 0                0.0
dest_airport_id                  0                0.0
dest                             0                0.0
dest_city_name                   0                0.0
crs_dep_time                     0                0.0
crs_arr_time                     0                0.0
dup                              0                0.0
crs_elapsed_time            

In [41]:
# duplicates...
len(model_df) - len(model_df.drop_duplicates())

0

* no duplicates apparently....

## datatype conversion: time and categorical data

In [42]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90938 entries, 1 to 99999
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype  
---  ------              --------------  -----  
 0   fl_date             90938 non-null  object 
 1   mkt_unique_carrier  90938 non-null  object 
 2   branded_code_share  90938 non-null  object 
 3   mkt_carrier         90938 non-null  object 
 4   mkt_carrier_fl_num  90938 non-null  int64  
 5   op_unique_carrier   90938 non-null  object 
 6   tail_num            90938 non-null  object 
 7   op_carrier_fl_num   90938 non-null  int64  
 8   origin_airport_id   90938 non-null  int64  
 9   origin              90938 non-null  object 
 10  origin_city_name    90938 non-null  object 
 11  dest_airport_id     90938 non-null  int64  
 12  dest                90938 non-null  object 
 13  dest_city_name      90938 non-null  object 
 14  crs_dep_time        90938 non-null  int64  
 15  crs_arr_time        90938 non-null  int64  
 16  dup 

In [43]:
test_df['fl_date']

0         2020-01-01 00:00:00
1         2020-01-01 00:00:00
2         2020-01-01 00:00:00
3         2020-01-01 00:00:00
4         2020-01-01 00:00:00
                 ...         
660551    2020-01-31 00:00:00
660552    2020-01-31 00:00:00
660553    2020-01-31 00:00:00
660554    2020-01-31 00:00:00
660555    2020-01-31 00:00:00
Name: fl_date, Length: 660556, dtype: object

In [44]:
model_df['fl_date'] = pd.to_datetime(model_df['fl_date'])

In [45]:
test_df.nunique()

fl_date                 31
mkt_unique_carrier      10
branded_code_share      15
mkt_carrier             10
mkt_carrier_fl_num    6725
op_unique_carrier       25
tail_num              5802
op_carrier_fl_num     6725
origin_airport_id      365
origin                 365
origin_city_name       358
dest_airport_id        365
dest                   365
dest_city_name         358
crs_dep_time          1242
crs_arr_time          1329
dup                      1
crs_elapsed_time       467
flights                  1
distance              1486
dtype: int64

In [46]:
model_df.nunique()

fl_date                730
mkt_unique_carrier      11
branded_code_share      16
mkt_carrier             11
mkt_carrier_fl_num    6667
op_unique_carrier       28
tail_num              6069
op_carrier_fl_num     6668
origin_airport_id      368
origin                 368
origin_city_name       361
dest_airport_id        367
dest                   367
dest_city_name         360
crs_dep_time          1241
crs_arr_time          1326
dup                      1
crs_elapsed_time       465
flights                  1
distance              1500
arr_delay              509
cancelled_flights        2
dtype: int64

In [48]:
#model_df[model_df['cancelled_flights'] != 0].shape

In [49]:
# change categorical columns to category type

categorical_columns = []
for col in model_df.columns[:-1]: # ignore arr_delay because it's not in test_df
    # check if both columns have unique values less than 30
    if model_df[col].nunique()<30 and test_df[col].nunique() < 30:
        categorical_columns.append(col)
        model_df[col] = model_df[col].astype('category')

categorical_columns        

['mkt_unique_carrier',
 'branded_code_share',
 'mkt_carrier',
 'op_unique_carrier',
 'dup',
 'flights']

In [50]:
model_df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90938 entries, 1 to 99999
Data columns (total 22 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   fl_date             90938 non-null  datetime64[ns]
 1   mkt_unique_carrier  90938 non-null  category      
 2   branded_code_share  90938 non-null  category      
 3   mkt_carrier         90938 non-null  category      
 4   mkt_carrier_fl_num  90938 non-null  int64         
 5   op_unique_carrier   90938 non-null  category      
 6   tail_num            90938 non-null  object        
 7   op_carrier_fl_num   90938 non-null  int64         
 8   origin_airport_id   90938 non-null  int64         
 9   origin              90938 non-null  object        
 10  origin_city_name    90938 non-null  object        
 11  dest_airport_id     90938 non-null  int64         
 12  dest                90938 non-null  object        
 13  dest_city_name      90938 non-null  object    

In [51]:
model_df[list(set(model_df.columns) - set(categorical_columns))]

,origin_city_name,dest_airport_id,origin_airport_id,crs_elapsed_time,dest,dest_city_name,crs_dep_time,op_carrier_fl_num,tail_num,arr_delay,origin,cancelled_flights,distance,mkt_carrier_fl_num,crs_arr_time,fl_date
1,"Columbia, SC",11057,10868,66.0,CLT,"Charlotte, NC",1949,4847,N646AE,0.0,CAE,1,88,4847,2055,2019-09-23
3,"Dallas/Fort Worth, TX",12191,11298,70.0,HOU,"Houston, TX",1630,5983,N959LR,0.0,DFW,1,247,5983,1740,2019-05-18
4,"Philadelphia, PA",11267,14100,116.0,DAY,"Dayton, OH",1540,5002,N625AE,0.0,PHL,1,477,5002,1736,2018-05-15
5,"San Jose, CA",14679,14831,75.0,SAN,"San Diego, CA",955,1140,N462WN,0.0,SJC,1,417,1140,1110,2019-02-27
6,"Newark, NJ",11995,11618,105.0,GSO,"Greensboro/High Point, NC",1000,4380,N13553,0.0,EWR,1,445,4380,1145,2019-06-29
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,"Washington, DC",12953,11278,76.0,LGA,"New York, NY",659,6134,N872RW,290.0,DCA,0,214,6134,815,2018-06-04
99996,"Dallas/Fort Worth, TX",13487,11298,152.0,MSP,"Minneapolis, MN",800,3909,N275SY,2.0,DFW,0,852,3909,1032,2019-06-23
99997,"Phoenix, AZ",14262,14107,70.0,PSP,"Palm Springs, CA",1745,3055,N778SK,12.0,PHX,0,261,3055,1755,2019-01-15
99998,"Houston, TX",11259,12191,60.0,DAL,"Dallas, TX",1530,38,N552WN,34.0,HOU,0,239,38,1630,2018-01-17


In [52]:
model_df[categorical_columns]

,mkt_unique_carrier,branded_code_share,mkt_carrier,op_unique_carrier,dup,flights
1,AA,AA_CODESHARE,AA,PT,N,1
3,AA,AA_CODESHARE,AA,YV,N,1
4,AA,AA_CODESHARE,AA,PT,N,1
5,WN,WN,WN,WN,N,1
6,UA,UA_CODESHARE,UA,EV,N,1
...,...,...,...,...,...,...
99995,DL,DL_CODESHARE,DL,YX,N,1
99996,DL,DL_CODESHARE,DL,OO,N,1
99997,AA,AA_CODESHARE,AA,OO,N,1
99998,WN,WN,WN,WN,N,1


In [53]:
sum(model_df['mkt_unique_carrier'].astype(str) == model_df['op_unique_carrier'].astype(str))

49485

In [54]:
sum(model_df['op_carrier_fl_num'] == model_df['mkt_carrier_fl_num'])

90935

In [55]:
model_df[model_df['op_carrier_fl_num'] != model_df['mkt_carrier_fl_num']]

,fl_date,mkt_unique_carrier,branded_code_share,mkt_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,op_carrier_fl_num,origin_airport_id,origin,...,dest,dest_city_name,crs_dep_time,crs_arr_time,dup,crs_elapsed_time,flights,distance,arr_delay,cancelled_flights
87744,2019-02-18,UA,UA_CODESHARE,UA,4015,YX,N642RW,3287,12266,IAH,...,ELP,"El Paso, TX",745,859,N,134.0,1,667,83.0,0
89326,2018-01-11,UA,UA_CODESHARE,UA,6048,YX,N653RW,3288,12266,IAH,...,MCI,"Kansas City, MO",1620,1824,N,124.0,1,643,125.0,0
94804,2019-01-15,DL,DL_CODESHARE,DL,5744,OO,N253SY,7335,13342,MKE,...,SLC,"Salt Lake City, UT",1834,2120,N,226.0,1,1247,16.0,0


### dropping unnecessary columns even though they're in test_df
- dup, flight
- dest_city_name, origin_city_name, dest_airport_id, origin_airport_id - dest and origin are better
- crs_elapsed_time not necessary since we have dep_time and arr_time

In [56]:
columns_to_drop = ['dup',
                  'flights',
                  'dest_city_name',
                  'origin_city_name',
                  'dest_airport_id',
                  'origin_airport_id',
                  'branded_code_share',
                  'mkt_carrier',
                  'op_carrier_fl_num'] 
model_df_2 = model_df.drop(columns=columns_to_drop)
model_df_2.head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,cancelled_flights
1,2019-09-23,AA,4847,PT,N646AE,CAE,CLT,1949,2055,66.0,88,0.0,1
3,2019-05-18,AA,5983,YV,N959LR,DFW,HOU,1630,1740,70.0,247,0.0,1
4,2018-05-15,AA,5002,PT,N625AE,PHL,DAY,1540,1736,116.0,477,0.0,1
5,2019-02-27,WN,1140,WN,N462WN,SJC,SAN,955,1110,75.0,417,0.0,1
6,2019-06-29,UA,4380,EV,N13553,EWR,GSO,1000,1145,105.0,445,0.0,1


In [57]:
model_df_2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 90938 entries, 1 to 99999
Data columns (total 13 columns):
 #   Column              Non-Null Count  Dtype         
---  ------              --------------  -----         
 0   fl_date             90938 non-null  datetime64[ns]
 1   mkt_unique_carrier  90938 non-null  category      
 2   mkt_carrier_fl_num  90938 non-null  int64         
 3   op_unique_carrier   90938 non-null  category      
 4   tail_num            90938 non-null  object        
 5   origin              90938 non-null  object        
 6   dest                90938 non-null  object        
 7   crs_dep_time        90938 non-null  int64         
 8   crs_arr_time        90938 non-null  int64         
 9   crs_elapsed_time    90938 non-null  float64       
 10  distance            90938 non-null  int64         
 11  arr_delay           90938 non-null  float64       
 12  cancelled_flights   90938 non-null  int64         
dtypes: category(2), datetime64[ns](1), float64(2),

In [58]:
# saving into folder

model_df_2.to_csv('data/model_df_version_1.csv', index=False)
pd.read_csv('data/model_df_version_1.csv').head()

,fl_date,mkt_unique_carrier,mkt_carrier_fl_num,op_unique_carrier,tail_num,origin,dest,crs_dep_time,crs_arr_time,crs_elapsed_time,distance,arr_delay,cancelled_flights
0,2019-09-23,AA,4847,PT,N646AE,CAE,CLT,1949,2055,66.0,88,0.0,1
1,2019-05-18,AA,5983,YV,N959LR,DFW,HOU,1630,1740,70.0,247,0.0,1
2,2018-05-15,AA,5002,PT,N625AE,PHL,DAY,1540,1736,116.0,477,0.0,1
3,2019-02-27,WN,1140,WN,N462WN,SJC,SAN,955,1110,75.0,417,0.0,1
4,2019-06-29,UA,4380,EV,N13553,EWR,GSO,1000,1145,105.0,445,0.0,1
